In [5]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
from tvDatafeed import TvDatafeed, Interval

In [6]:
def getdata(name):
    data=tv.get_hist(symbol=name,exchange='BINANCE',interval=Interval.in_daily,n_bars=5000)
    data=data.reset_index()
    data["datetime"]=data["datetime"].dt.strftime('%Y-%m-%d')
    data2=data[data["datetime"]>="2020-01-01"].copy()
    data2["datetime"] = pd.to_datetime(data2["datetime"])
    return data2

In [7]:
def getapi(api):
    response_API=requests.get(api)
    data=response_API.text
    parse_json=json.loads(data)
    df = pd.json_normalize(parse_json['values'])
    listtime=list(df['x'])
    listtime2=[]
    for i in listtime:
        listtime2.append(datetime.datetime.fromtimestamp(i).strftime('%Y-%m-%d'))
    df['datetime']=pd.Series(listtime2)
    df["datetime"] = pd.to_datetime(df["datetime"])
    df=df[df["datetime"]>="2020-01-01"].copy()
    df=df.drop(columns=['x'])
    df.rename(columns={'y':'value'},inplace=True)
    df=df.drop_duplicates(subset=['datetime'])
    return df

In [8]:
tv = TvDatafeed()
bitcoi=getdata('BTCUSDT')
eth=getdata('ETHUSDT')
bnb=getdata('BNBUSDT')
ada=getdata('ADAUSDT')
bitcoi=bitcoi[["datetime","open","close"]]
bitcoi.tail(5)

you are using nologin method, data you access may be limited


,datetime,open,close
1649,2022-02-20,38386.89,37008.16
1650,2022-02-21,37008.16,38230.33
1651,2022-02-22,38230.33,37250.01
1652,2022-02-23,37250.02,38327.21
1653,2022-02-24,38328.68,39082.87


In [9]:
number_transactions=getapi('https://api.blockchain.info/charts/n-transactions?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
number_address=getapi('https://api.blockchain.info/charts/n-unique-addresses?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
transaction_second=getapi('https://api.blockchain.info/charts/transactions-per-second?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
transaction_second=transaction_second.groupby(['datetime']).mean().reset_index()
total_bitcoin=getapi('https://api.blockchain.info/charts/total-bitcoins?timespan=4years&rollingAverage=24hours&format=json&sampled=true')
hash_rate=getapi('https://api.blockchain.info/charts/hash-rate?timespan=4years&rollingAverage=24hours&format=json&sampled=false')

In [10]:
bitcoin=bitcoi.merge(eth[["datetime","open"]],on='datetime',how='left',suffixes=('_bitcoin', '_eth'))
bitcoin=bitcoin.merge(bnb[["datetime","open"]],on='datetime',how='left')
bitcoin=bitcoin.merge(ada[["datetime","open"]],on='datetime',how='left',suffixes=('_bnb', '_ada'))

In [11]:
data_api=number_transactions.merge(number_address,on='datetime',how='left',suffixes=('_number_transaction', '_number_address'))
data_api=data_api.merge(transaction_second,on='datetime',how='left')
data_api=data_api.merge(total_bitcoin,on='datetime',how='left',suffixes=('_transaction_second', '_total_bitcoin'))
data_api=data_api.merge(hash_rate,on='datetime',how='left')
data_api.rename(columns={'value':'value_hash_rate'},inplace=True)

In [12]:
bitcoin=bitcoin.merge(data_api,on='datetime',how='left')
bitcoin.rename(columns={'close':'close_bitcoin'},inplace=True)
bitcoin.tail(5)

,datetime,open_bitcoin,close_bitcoin,open_eth,open_bnb,open_ada,value_number_transaction,value_number_address,value_transaction_second,value_total_bitcoin,value_hash_rate
781,2022-02-20,38386.89,37008.16,2621.99,380.7,0.931,237402.0,615656.0,2.509896,1.896024e+07,1.710013e+08
782,2022-02-21,37008.16,38230.33,2568.03,356.7,0.856,260265.0,647191.0,2.343229,1.896112e+07,2.113187e+08
783,2022-02-22,38230.33,37250.01,2636.28,374.3,0.888,251275.0,622682.0,2.879340,1.896200e+07,2.085382e+08
784,2022-02-23,37250.02,38327.21,2579.70,365.6,0.866,254139.0,676575.0,2.958160,1.896289e+07,1.835136e+08
785,2022-02-24,38328.68,39082.87,2596.26,361.2,0.852,NaN,NaN,NaN,NaN,NaN


In [13]:
bitcoin.to_pickle("../data/interim/bitcoin.pkl")